In [430]:
#STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type 
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [431]:
import pandas as pd
import numpy as np
import datetime

In [432]:
tvl = pd.read_csv('csv_outputs/op_summer_latest_stats.csv')
distrib_df = pd.read_csv('csv_outputs/dune_op_distribution_type.csv')
program_df = pd.read_csv('inputs/OP Summer Programs [Public].csv')

In [433]:
#Filter TVL DF
tvl = tvl[tvl['include_in_summary'] == 1]
tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
# display(tvl)

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_97339/4163294631.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl['join_key'] = tvl['top_level_name'].str.replace('*','') #tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')


In [434]:
op_token_columns = ['op_claimed','op_deployed', 'op_from_other_projects','op_to_other_projects','op_to_project']

In [435]:
#Set up Distributions for Mapping
distrib_df['program_map'] = np.where(
                                distrib_df['to_type'].isin(tvl['top_level_name'])
                                ,distrib_df['to_type']
                                ,''
                                )
group_cols = ['from_name','to_label','to_type','program_map'] + op_token_columns

sum_distrib_df = distrib_df[group_cols].groupby(['from_name','program_map']).sum()
sum_distrib_df.reset_index(inplace=True)
#Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df['join_key'] = np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])
                            # sum_distrib_df['from_name'] + ' - ' \
                            #     + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

# display(sum_distrib_df[sum_distrib_df['join_key'].str.contains('elodr')])

/var/folders/xb/s7wz9wjj1x9b5w92rvtsqw_80000gn/T/ipykernel_97339/15182629.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum_distrib_df = distrib_df[group_cols].groupby(['from_name','program_map']).sum()


In [436]:
#lowercase joinkeys
tvl['join_key'] = tvl['join_key'].str.lower()
sum_distrib_df['join_key'] = sum_distrib_df['join_key'].str.lower()
#
df = sum_distrib_df.merge(tvl, on='join_key',how='outer')
# display(df[df['join_key'].str.contains('velodr')])

In [437]:
# Overrides as needed
def replace_program_names(df, overrides):
    for program, program_override in overrides.items():
        df.loc[df['join_key'] == program, 'from_name'] = program_override
    return df

#Overrides if needed
overrides = {
    'old name'  :   'new name',
}

# Replace program names with overrides
df = replace_program_names(df, overrides)

In [438]:
#Create the aggregate app name field
df['agg_app_name'] = df['app_name'].combine_first(df['from_name'])
df = df.fillna(0) #Fill NA with 0
# display(df)
display(df[df['agg_app_name'].str.contains('elodr')])

,from_name,program_map,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,join_key,date,include_in_summary,...,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,flows_retention,last_price_net_dollar_flows_retention,agg_app_name
103,Velodrome,,4327776.0,4400739.0,47272.0,0.0,6999999.0,velodrome,2023-03-11,1.0,...,Gov & Partner Fund,2022-07-13,0,2.395393e+08,2.395393e+08,2.395393e+08,2.395393e+08,0.0,0.0,Velodrome


In [439]:
# Now union back again

data_cols = ['agg_app_name','top_level_name', 'program_name',\
               'num_op_override', 'period','op_source', 'start_date','end_date',
               'cumul_net_dollar_flow_at_program_end',
                'cumul_net_dollar_flow',
                'cumul_last_price_net_dollar_flow_at_program_end',
                'cumul_last_price_net_dollar_flow'
                ]
select_cols = data_cols + op_token_columns

group_cols = select_cols[:8] #group by 1 to 8
print(group_cols)

# display(df[select_cols])

sum_distrib_df = df[select_cols].groupby(group_cols).sum()
sum_distrib_df.reset_index(inplace=True)

['agg_app_name', 'top_level_name', 'program_name', 'num_op_override', 'period', 'op_source', 'start_date', 'end_date']


In [440]:
# #Get the rank by start_date of each program

# sum_distrib_df = sum_distrib_df.reset_index().rename(columns={'index': 'row_num'})

# # replace 0s with '9999-12-31'
# sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == 0, '9999-12-31', sum_distrib_df['start_date'])
# # create a new column 'program_rank' based on the 'start_date' column
# sum_distrib_df = sum_distrib_df.sort_values(['agg_app_name', 'start_date', 'row_num'])
# sum_distrib_df['program_rank'] = sum_distrib_df.groupby('agg_app_name').cumcount() + 1

# sum_distrib_df = sum_distrib_df.sort_values(by=['agg_app_name','program_rank'],ascending=[True,True])
# # subtract all overridden values fromthe amount I have deployed

# # create a new column 'cumulative_num_op_override' that contains the cumulative sum of 'num_op_override' for each agg_app_name group
# sum_distrib_df['cumulative_num_op_override'] = sum_distrib_df.groupby('agg_app_name')['num_op_override'].cumsum()


# # create a new column 'op_deployed_net_override' that subtracts 'cumulative_num_op_override' from 'op_deployed'
# sum_distrib_df['op_deployed_net_override'] = sum_distrib_df['op_deployed'] - sum_distrib_df.groupby('agg_app_name')['cumulative_num_op_override'].shift(1).fillna(0)
# # drop the 'cumulative_num_op_override' column
# # sum_distrib_df.drop('cumulative_num_op_override', axis=1, inplace=True)

# # replace '9999-12-31' with 0s
# sum_distrib_df['start_date'] = np.where(sum_distrib_df['start_date'] == '9999-12-31', 0, sum_distrib_df['start_date'])

# #Drop Row Num
# sum_distrib_df.drop('row_num', axis=1, inplace=True)

In [441]:
# Override # OP Deployed 
sum_distrib_df['op_deployed'] = sum_distrib_df.apply(lambda x: x['num_op_override'] if x['num_op_override'] > 0 else x['op_deployed'], axis=1)

# # replace 0s in 'num_op_override' with the corresponding value in 'op_deployed_net_override'
# sum_distrib_df['og_op_deployed'] = sum_distrib_df['op_deployed'] 
# sum_distrib_df['op_deployed'] = np.where(sum_distrib_df['num_op_override'] == 0, sum_distrib_df['op_deployed_net_override'], sum_distrib_df['num_op_override'])

# #Select all except the last 4 rows
# sum_distrib_df = sum_distrib_df.iloc[:, :-4]


In [442]:
# latest tvl metrics
sum_distrib_df['net_flows_retention'] = np.where(sum_distrib_df['period']=='Post-Program'\
            ,sum_distrib_df['cumul_last_price_net_dollar_flow'] / sum_distrib_df['cumul_last_price_net_dollar_flow_at_program_end']\
            ,np.nan\
)
# If < 0 then make retention 0
sum_distrib_df['net_flows_retention'] = np.where(sum_distrib_df['cumul_last_price_net_dollar_flow']<0\
            ,0\
            ,sum_distrib_df['net_flows_retention']\
)

In [443]:
display(sum_distrib_df)
sum_distrib_df['last_updated'] = pd.to_datetime(datetime.datetime.now())
sum_distrib_df.to_csv('csv_outputs/incentives_stats_summary.csv')
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])

,agg_app_name,top_level_name,program_name,num_op_override,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,net_flows_retention
0,1inch,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,25359.0,300000.0,0.0,0.0,300000.0,0.000000e+00
1,Aave,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,300000.0,0.000000e+00
2,Aave,Aave - Liquidity Mining,Aave - Aave - Liquidity Mining,5000000.0,Post-Program,Partner Fund,2022-08-04,2022-11-04,3.450056e+08,7.849483e+07,3.419694e+08,7.732963e+07,4820532.0,5000000.0,0.0,0.0,0.0,7.732963e+07
3,Aelin,Aelin,Aelin,0.0,Post-Program,Gov Fund - Phase 0,2022-09-12,2022-09-14,-2.827652e+03,-2.778652e+03,-2.826693e+03,-2.777707e+03,880175.0,900002.0,0.0,19827.0,900000.0,0.000000e+00
4,Alchemix,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,21722.0,10397.0,0.0,0.0,250000.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Xtoken,Xtoken,Xtoken,0.0,During Program,Gov Fund - Season 1,2022-09-19,0,1.365099e+06,1.365099e+06,1.596840e+06,1.596840e+06,0.0,0.0,0.0,0.0,0.0,1.596840e+06
107,Yearn,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,1000000.0,0.000000e+00
108,Zeroex (0x),0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,452.0,452.0,0.0,0.0,300000.0,0.000000e+00
109,Zipswap,0,0,0.0,0,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2469.0,2469.0,0.0,0.0,26.0,0.000000e+00
